# Basis for comparision of 2 models
* The model metrics i.e how big and "fast" the model is, constitute the model's metrics in this notebook.
  * How Big?  Space Complexity
    - Filesize of the ".p" file of a frozen model
    - Size of state_dict_object (with or without the trained weights)
  * How fast? Time Complexity
    - MACS and FLOPs
    - Forward pass timing `as this only constitutes the Inference time when deployed`



In [33]:
import torch
import torch.nn as nn
import numpy as np


In [34]:
from torchvision.models import resnet18, resnet50, resnet101

In [35]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # Device is a constant through the nb
DATA = torch.randn( size = (3, 3, 224, 224) )


# we create al list of models to compare
model_list = [ resnet18(pretrained = False).to(DEVICE),\
               resnet50(pretrained = False).to(DEVICE),
               resnet101(pretrained = False).to(DEVICE)
             ]



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# How Big the model is?

Model size consists of 2 things:-
- Parameters ( model.parameters() )
- Buffers ( model.buffers() )
- Each paramter/buffer has 2 methods
  * p.nelement()
  * p.element_size()

- Using these info can we get the size of each model?

In [36]:

# Observe:
# The number of learnable-params and file_size of the saved models should be equal

def get_metrics_on_device(model: nn.Module):
  '''
    The learnable params conatins 2 things
      - Parameters
      - Buffers
    We can get the number of params/buffers and size of each param/buffer from the methods
    pertaining to each param and buffer
      - ele.nelements(), ele.element_size()
  '''

  parameters_size = sum([ elem.element_size() * elem.nelement()  for elem in model.parameters() ])
  buffers_size = sum([ elem.element_size() * elem.nelement()  for elem in model.buffers() ])

  total_size_on_device = parameters_size + buffers_size
  return total_size_on_device / (1024 * 1024) # to mega

model_names = ["resnet18", "resnet50", "resnet101"]
for i, model in enumerate(model_list):
  size_of_model_on_device = get_metrics_on_device(model)
  print(f"Size of {model_names[i]} on device is: {size_of_model_on_device:.3f} MB")
print()
print()





def get_size_of_network_on_disk(model: nn.Module):
  '''
    1. Get the state_dict_object
    2. Make a copy of it and save it to disk
    3. Get the filesize
    4. Remove the file
    5. Return the filesize
  '''
  import os

  state_dict_object = model.state_dict()
  file_path = "model.p"
  torch.save(obj = state_dict_object, f = file_path)

  file_size = os.path.getsize(file_path)
  os.remove(file_path)

  return file_size / (1024 * 1024)

model_names = ["resnet18", "resnet50", "resnet101"]
for i, model in enumerate(model_list):
  size_of_model_on_disk = get_size_of_network_on_disk(model)
  print(f"Size of {model_names[i]} on disk is: {size_of_model_on_disk:.3f} MB")




Size of resnet18 on device is: 44.629 MB
Size of resnet50 on device is: 97.695 MB
Size of resnet101 on device is: 170.344 MB


Size of resnet18 on disk is: 44.661 MB
Size of resnet50 on disk is: 97.778 MB
Size of resnet101 on disk is: 170.507 MB


# How Fast: Model Metrics
1. MAC: `1 addition + 1 multiplication`
2. FLOPs: `Total number of additions and multiplications`
3. Inference time

**FLOPs = 2 x MAC**  

In [41]:
!wget https://optimization-thinkautonomous.s3.eu-west-3.amazonaws.com/thop_library.zip && unzip -q thop_library.zip

--2024-05-30 06:04:38--  https://optimization-thinkautonomous.s3.eu-west-3.amazonaws.com/thop_library.zip
Resolving optimization-thinkautonomous.s3.eu-west-3.amazonaws.com (optimization-thinkautonomous.s3.eu-west-3.amazonaws.com)... 3.5.226.246, 52.95.156.60
Connecting to optimization-thinkautonomous.s3.eu-west-3.amazonaws.com (optimization-thinkautonomous.s3.eu-west-3.amazonaws.com)|3.5.226.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74072 (72K) [application/zip]
Saving to: ‘thop_library.zip.2’

thop_library.zip.2  100%[===================>]  72.34K   358KB/s    in 0.2s    

2024-05-30 06:04:38 (358 KB/s) - ‘thop_library.zip.2’ saved [74072/74072]

replace __MACOSX/._thop_library? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### MACs, FLOPs and number of Parameters

In [42]:

# thop_library is a cool way to calc these
import sys
sys.path.append('thop_library') # so that interpreter points to this.
from thop import profile



def get_metrics(model: nn.Module, data: torch.Tensor):
  '''
    MACs and FLOPs
  '''
  MACs, parameters = profile( model=model, inputs = (data, ), verbose=False ) # verbose = False is imp
  FLOPs = 2*MACs

  return MACs/(1e6), FLOPs/1e6, parameters/1e6


for i, model in enumerate(model_list):
  MMACs, MFLOPs, MParams = get_metrics(model.to(DEVICE), DATA.to(DEVICE))
  print( f"MACs, FLOPs and Parameters for {model_names[i]} are: {MMACs:.3f} Mega units, {MFLOPs:.3f} Mega units, {MParams:.3f} Mega units " )





MACs, FLOPs and Parameters for resnet18 are: 5457.199 Mega units, 10914.398 Mega units, 11.690 Mega units 
MACs, FLOPs and Parameters for resnet50 are: 12334.544 Mega units, 24669.088 Mega units, 25.557 Mega units 
MACs, FLOPs and Parameters for resnet101 are: 23501.915 Mega units, 47003.830 Mega units, 44.549 Mega units 


### Inference Time

In [59]:
def update_device(device):
  if(device != "cpu" and torch.cuda.is_available() ):
    return device
  return "cpu" # either gpu not available of explicitly cpu

def get_average_inference_time(model: nn.Module, data: torch.Tensor, niters: int, device: str):
  '''
    1. Get the total num of datapoints across all iterations
    2. Set the start time
    3. Inference Pipeline
    4. Set the end time and calc the time taken
  '''
  import time

  device = update_device(device)

  batch_size = data.shape[0]
  num_datapoints = batch_size * niters

  model = model.to(device)
  data = data.to(device)

  total_time = np.inf
  if(niters == 0):
      return total_time

  model.eval()

  with torch.no_grad(): # Dont track the gradients and make it faster
    start_time = time.time()
    for iter in range(niters):
        model(data)

    end_time = time.time()

    total_time = end_time - start_time
  return ( total_time / num_datapoints ), device


for i, model in enumerate(model_list):
  avg_time, device = get_average_inference_time(model, DATA, 10, DEVICE)
  print( f"Average Inference Time of {model_names[i]} is: {avg_time:.3f} seconds " )

Average Inference Time of resnet18 is: 0.001 seconds 
Average Inference Time of resnet50 is: 0.003 seconds 
Average Inference Time of resnet101 is: 0.005 seconds 
